In [1]:
include("BayesianUtilities.jl")
using Distributions, LinearAlgebra, .BayesianUtilities

In [2]:
logp(z) = logpdf(Normal(2.1, 3.4),z)
q = Normal(1.5, 0.8)

cvi(logp,q)

2-element Array{Float64,1}:
  0.2008467229653198
 -0.043252595155709346

In [3]:
cvi(logp,q,10)

2-element Array{Float64,1}:
  0.16979063770999409
 -0.043252595155709346

In [4]:
logp(z) = logpdf(MvNormal([2.1, 1.0], [3.4 1; 1 1.5]),z)
q = MvNormal([1.5,0.2], diagm(0=>ones(2)))

cvi(logp,q)

6-element Array{Float64,1}:
 -0.21268752509697114
  2.1993331693807567
 -0.18292682926829262
  0.12195121951219506
  0.12195121951219506
 -0.4146341463414633

In [26]:
z1 = Normal(1,2)
z3 = Gamma(1,2)
z4 = Normal(2,3)

f(z1,z2,z3,z4) = logpdf(Normal(z1*z2,1/sqrt(z3)),z4);

In [33]:
using Zygote, ForwardDiff

#df(s1,s2,s3,s4) = Zygote.forwarddiff(s1 -> f(s1,s2,s3,s4), s1);
#ddf(s1,s2,s3,s4) = Zygote.forwarddiff(s1 -> df(s1,s2,s3,s4), s1);
df(s1,s2,s3,s4) = ForwardDiff.derivative(s1 -> f(s1,s2,s3,s4), s1);
ddf(s1,s2,s3,s4) = ForwardDiff.derivative(s1 -> df(s1,s2,s3,s4), s1);
h__(s1,s2,s3,s4) = s1*ddf(s1,s2,s3,s4)
g_(s1,s2,s3,s4) = s1*(df(s1,s2,s3,s4)-h__(s1,s2,s3,s4))
h_(s1,s2,s3,s4) = 0.5*s1^2*ddf(s1,s2,s3,s4)
l(s1,s2,s3,s4) = f(s1,s2,s3,s4) - g_(s1,s2,s3,s4) - h_(s1,s2,s3,s4)
s_3, s_4 = rand(z3), rand(z4)
println(l(-5,1,s_3,s_4))
println(l(2,1,s_3,s_4))
g(s1,s2,s3,s4) = mean(s1)*(df(s1,s2,s3,s4)-h__(s1,s2,s3,s4))
h(s1,s2,s3,s4) = 0.5*(mean(s1)^2+var(s1))*ddf(s1,s2,s3,s4)
g_new(s2,s3,s4) = g(1e-15,s2,s3,s4)
h_new(s2,s3,s4) = h(1e-15,s2,s3,s4)
f_new(s2,s3,s4) = l(1e-15,s2,s3,s4) + g_new(s2,s3,s4) + h_new(s2,s3,s4)

-2.118287590990694
-2.118287590990695


f_new (generic function with 1 method)

In [35]:
df(1,2,2,3)

4.000000000000001

In [3]:
g = message(f,[z1,nothing,z3,z4])

#90 (generic function with 1 method)

In [4]:
g(1,1,1)

-1.4189385332046704

In [5]:
l = message(f,z1,nothing,z3,z4)

-1368.304041585266
-2.4142630515044523


(::Main.BayesianUtilities.var"#l#81"{typeof(f),Main.BayesianUtilities.var"#g_#79"{Main.BayesianUtilities.var"#df#74"{typeof(f)},Main.BayesianUtilities.var"#h__#78"{Main.BayesianUtilities.var"#ddf#76"{Main.BayesianUtilities.var"#df#74"{typeof(f)}}}},Main.BayesianUtilities.var"#h_#80"{Main.BayesianUtilities.var"#ddf#76"{Main.BayesianUtilities.var"#df#74"{typeof(f)}}}}) (generic function with 1 method)

In [6]:
l(1,2,3)

-15.819804555680891

In [12]:
@eval [$(Symbol("s_$i")) for $i=1:5]

LoadError: UndefVarError: i not defined

In [14]:
args = [:x, :y, :z];
:(f(1, $(args...)))

:(f(1, x, y, z))

In [18]:
g($(args...)) = eval(:(f(1, $(args...))))

LoadError: syntax: "$(args...)" is not a valid function argument name around In[18]:1

In [17]:
g(1,2,3)

LoadError: UndefVarError: x not defined

In [2]:
p1 = MvNormal([1,1],diagm(0=>ones(2)))
p2 = MvNormal([-1,-1],diagm(0=>ones(2)))
W1 = Wishart(2,diagm(0=>ones(2)))
W2 = Wishart(2,diagm(0=>ones(2)))
m_z = normalmix([1,0.8],nothing,[p1,p2],[W1,W2])

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(2)
p: [0.011619242357096736, 8.674761447897611e-6]
)


In [3]:
pz = collide(Categorical(2), m_z)

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(2)
p: [0.9992539711661633, 0.0007460288338366962]
)


In [4]:
normalmix([1,0.8],pz,nothing,[W1,W2])

2-element Array{MvNormal,1}:
 FullNormal(
dim: 2
μ: [1.0, 0.8]
Σ: [0.49962698558308166 0.0; 0.0 0.49962698558308166]
)

 FullNormal(
dim: 2
μ: [1.0, 0.8]
Σ: [0.0003730144169183481 0.0; 0.0 0.0003730144169183481]
)


In [5]:
normalmix([1,0.8],pz,[p1,p2],nothing)

[[-0.49962698558308166, 0.0, 0.0, -0.519612065006405, 0.49962698558308166], [-0.0018650720845917405, -0.0013428519009060533, -0.0013428519009060533, -0.001581581127733796, 0.0003730144169183481]]


2-element Array{Wishart{Float64,PDMats.PDMat{Float64,Array{Float64,2}},Int64},1}:
 Wishart{Float64,PDMats.PDMat{Float64,Array{Float64,2}},Int64}(
df: 2.0
S: [1.0007465858083766 0.0; 0.0 0.9622563325080543]
)

 Wishart{Float64,PDMats.PDMat{Float64,Array{Float64,2}},Int64}(
df: 2.0
S: [689.7362185718857 -585.6250912402803; -585.6250912402803 813.3681822781671]
)


In [2]:
c = Canonical(Normal,[0,-1])

Canonical(Normal, [0, -1])

In [3]:
c.η

2-element Array{Int64,1}:
  0
 -1

In [4]:
c.dist

Normal

In [5]:
c::Distribution

LoadError: TypeError: in typeassert, expected Distribution, got a value of type Canonical

In [6]:
Normal{Float64}<:Normal

true

In [7]:
collide(c,c)

Canonical(Normal, [0, -2])

In [8]:
2\Normal(3.4,sqrt(4))

Normal{Float64}(μ=1.7, σ=1.0)

In [9]:
p = MvNormal([1.3,-1.5],[2.5 1.9;1.9 2.5])

FullNormal(
dim: 2
μ: [1.3, -1.5]
Σ: [2.5 1.9; 1.9 2.5]
)


In [10]:
A = [1 1; 0 1]

2×2 Array{Int64,2}:
 1  1
 0  1

In [11]:
A\p

FullNormal(
dim: 2
μ: [2.8, -1.5]
Σ: [1.2000000000000002 -0.6000000000000001; -0.6000000000000001 2.5]
)


In [12]:
A = [1 1 0; 0 1 1]
A\p

ξ = [2.3106060606060606, -0.04545454545454547, -2.356060606060606]
W = [0.9469696969696969 0.2272727272727273 -0.7196969696969696; 0.2272727272727273 0.4545454545454546 0.2272727272727273; -0.7196969696969696 0.2272727272727273 0.9469696969696969]


Canonical(MvNormal, [2.3106060606060606, -0.04545454545454547, -2.356060606060606, -0.47348484848484845, -0.11363636363636365, 0.3598484848484848, -0.11363636363636365, -0.2272727272727273, -0.11363636363636365, 0.3598484848484848, -0.11363636363636365, -0.47348484848484845])

In [13]:
p2 = MvNormal([1.3,-1.5,0.6],[2.5 1.9 1.4;1.9 4.3 0.3; 1.4 0.3 2.2])

FullNormal(
dim: 3
μ: [1.3, -1.5, 0.6]
Σ: [2.5 1.9 1.4; 1.9 4.3 0.3; 1.4 0.3 2.2]
)


In [14]:
A = [1 1; 0 1; 0 1]
A\p2

FullNormal(
dim: 2
μ: [1.5542372881355933, -0.07627118644067743]
Σ: [0.9237288135593222 -0.027118644067796578; -0.027118644067796578 1.588135593220339]
)


In [15]:
a = [1, 0]
a\p

Normal{Float64}(μ=2.44, σ=1.056)

In [16]:
a = [1 0]
a \ Normal(3.4,sqrt(4))

Canonical(MvNormal, [0.85, 0.0, -0.125, -0.0, -0.0, -0.0])

In [2]:
m1, m2 = Normal(0,2), Normal(2,1)
w1, w2 = Gamma(2,0.5), Gamma(3,1)
vmp = normalmix(2.4,nothing,[m1,m2],[w1,w2])

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(2)
p: [0.002647551691759494, 0.11107536260545436]
)


In [3]:
qz = collide(Categorical(0.5,0.5),vmp)

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(2)
p: [0.023280723222060075, 0.9767192767779399]
)


In [4]:
vmp = normalmix(2.4,qz,nothing,[w1,w2])

2-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=2.4, σ=6.55392951148153)
 Normal{Float64}(μ=2.4, σ=0.5841905037515628)

In [5]:
collide.(vmp,[m1,m2])

2-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=0.2044554591124335, σ=1.912914243106386)
 Normal{Float64}(μ=2.2982229169239177, σ=0.5044231434918559)

In [6]:
vmp = normalmix(2.4,qz,[m1,m2],nothing)

2-element Array{Gamma{Float64},1}:
 Gamma{Float64}(α=1.01164036161103, θ=0.11360992932365316)
 Gamma{Float64}(α=1.48835963838897, θ=0.5664971805312052)

In [8]:
collide.(vmp,[w1,w2])

2-element Array{Gamma{Float64},1}:
 Gamma{Float64}(α=2.01164036161103, θ=10.802047549481257)
 Gamma{Float64}(α=3.48835963838897, θ=2.7652338517595068)

In [2]:
collide(Dirichlet([1.5, 4.5, 6]), categorical([1,0,0],nothing), categorical([1,0,0],nothing))

Dirichlet{Float64}(
alpha: [3.5, 4.5, 6.0]
)


In [5]:
categorical(nothing,Dirichlet([2.5, 4.5, 6]))

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(3)
p: [0.1750843735769453, 0.3475756288047549, 0.4773399976182998]
)


In [50]:
collide(sqrt(1/0.2) * normal(0,nothing,1),sqrt(1/0.4) *normal(2,nothing,1))

Normal{Float64}(μ=2.1081851067789197, σ=1.2909944487358058)

In [6]:
collide(Normal(0,1),Normal(2,1))

Normal{Float64}(μ=1.0, σ=0.7071067811865476)

In [7]:
0.7071067811865476^2

0.5000000000000001

In [24]:
collide(Normal(0,1),Normal(2,2))

Normal{Float64}(μ=0.4, σ=0.8944271909999159)

In [38]:
collide(Normal(0,sqrt(1/0.2)),Normal(2,sqrt(1/0.4)))

Normal{Float64}(μ=1.3333333333333333, σ=1.2909944487358058)

In [43]:
collide(Normal(0,sqrt((2-1.33)/0.2)),Normal(2,sqrt((2-1.33)/0.4)))

Normal{Float64}(μ=1.3333333333333333, σ=1.0567244989431572)

In [40]:
collide(Normal(0,sqrt(2/0.2)),Normal(2,sqrt(2/0.4)))

Normal{Float64}(μ=1.3333333333333333, σ=1.8257418583505538)

In [11]:
collide(exp(0.2)*Normal(0,1),exp(0.4)*Normal(2,1))

Normal{Float64}(μ=1.1973753202249038, σ=0.9450592772566271)

In [12]:
0.8/0.6

1.3333333333333335

In [36]:
sqrt((0.4*(0-1.33)^2 + 0.2*(2-1.33)^2)/0.6)

1.1527792503337315

In [21]:
mm = MixtureModel(Normal[
   Normal(-2.0, 1.2),
   Normal(0.0, 1.0),
   Normal(3.0, 2.5)], [0.2, 0.5, 0.3])

MixtureModel{Normal}(K = 3)
components[1] (prior = 0.2000): Normal{Float64}(μ=-2.0, σ=1.2)
components[2] (prior = 0.5000): Normal{Float64}(μ=0.0, σ=1.0)
components[3] (prior = 0.3000): Normal{Float64}(μ=3.0, σ=2.5)


In [22]:
mean(mm)

0.4999999999999999

In [23]:
var(mm)

5.913

In [20]:
mm = MixtureModel(Normal[
   Normal(-2.0, 1.2),
   Normal(0.0, 1.0),
   Normal(3.0, 2.5)], [0.2, 0.8, 0.3])

LoadError: ArgumentError: Categorical: the condition isprobvec(p) is not satisfied.

In [3]:
m1, m2 = Normal(0,2), Normal(2,1)
w1, w2 = Gamma(2,0.5), Gamma(3,1)
vmp = normalmix(2.4,nothing,[m1,m2],[w1,w2])

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(2)
p: [0.002647551691759494, 0.11107536260545436]
)


In [4]:
collide(Categorical(0.5,0.5),vmp)

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(2)
p: [0.023280723222060075, 0.9767192767779399]
)


In [5]:
d = Dirichlet([1.5, 4.5, 6])
mean(d)

3-element Array{Float64,1}:
 0.125
 0.375
 0.5

In [6]:
Categorical(mean(d))

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(3)
p: [0.125, 0.375, 0.5]
)


In [10]:
c = Categorical([0.2,0.5,0.7],check_args=false)

DiscreteNonParametric{Int64,Float64,Base.OneTo{Int64},Array{Float64,1}}(
support: Base.OneTo(3)
p: [0.2, 0.5, 0.7]
)


In [12]:
c.p

3-element Array{Float64,1}:
 0.2
 0.5
 0.7

In [2]:
poisson(4, nothing)

Gamma{Float64}(α=5.0, θ=1.0)

In [3]:
collide(Gamma(3.2,5.5),poisson(4, nothing))

Gamma{Float64}(α=7.2, θ=1.1818181818181819)

In [4]:
normal(3.4, 2.5, nothing)

Gamma{Float64}(α=1.5, θ=0.40499999999999936)

In [5]:
normal(3, 2, nothing)

Gamma{Float64}(α=1.5, θ=0.5)

In [6]:
normal(3.4, nothing, 1)

Normal{Float64}(μ=3.4, σ=1.0)

In [7]:
mean(Gamma(3.2,5.5))

17.6

In [8]:
normal(3.4, nothing, Gamma(3.8,5.5))

Normal{Float64}(μ=3.3999999999999995, σ=0.21873931962990356)

In [9]:
normal(3, nothing, Gamma(3.8,5.5))

Normal{Float64}(μ=3.0, σ=0.21873931962990356)

In [12]:
a = 2
x = Normal(1,2)
normal(3.4, a*x, nothing)

Gamma{Float64}(α=1.5, θ=8.98)

In [10]:
dump(:(mean(x^2)))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol mean
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Symbol x
        3: Int64 2


In [11]:
function f(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end

f (generic function with 1 method)

In [12]:
f(:^,:x,2)

:(x ^ 2)

In [22]:
function f(op)
    expr = Expr(:call, op)
    return expr
end

expr = f(:(x^2))

:((x ^ 2)())

In [23]:
dump(expr)

Expr
  head: Symbol call
  args: Array{Any}((1,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Symbol x
        3: Int64 2


In [27]:
expr.args

1-element Array{Any,1}:
 :(x ^ 2)

In [30]:
dump(expr.args)

Array{Any}((1,))
  1: Expr
    head: Symbol call
    args: Array{Any}((3,))
      1: Symbol ^
      2: Symbol x
      3: Int64 2


In [20]:
canonform(Gamma(3,4))

LoadError: MethodError: no method matching canonform(::Gamma{Float64})
Closest candidates are:
  canonform(!Matched::Normal) at /Users/sakbayrak/.julia/packages/Distributions/nAJyo/src/univariate/continuous/normalcanon.jl:32
  canonform(!Matched::MvNormal{T,C,FillArrays.Zeros{T,N,Axes} where Axes where N}) where {C, T<:Real} at /Users/sakbayrak/.julia/packages/Distributions/nAJyo/src/multivariate/mvnormalcanon.jl:152
  canonform(!Matched::MvNormal{T,C,var"#s259"} where var"#s259"<:AbstractArray{T,1}) where {C, T<:Real} at /Users/sakbayrak/.julia/packages/Distributions/nAJyo/src/multivariate/mvnormalcanon.jl:148

In [22]:
cf(Gamma(3,4),4.2)

-3.721884091117499e-5 - 0.00020645392977038888im

In [25]:
mgf(Gamma(3,4),4)

-0.0002962962962962963

In [11]:
canonform(Normal())

NormalCanon{Float64}(η=0.0, λ=1.0, μ=0.0)

In [14]:
convolve(Gamma(3,4),Gamma(2,4))

Gamma{Float64}(α=5.0, θ=4.0)

In [19]:
convolve(Normal(5,sqrt(3)),Normal())

Normal{Float64}(μ=5.0, σ=2.0)

In [11]:
f(x,-) = x^2

f (generic function with 1 method)

In [12]:
f(3)

LoadError: MethodError: no method matching f(::Int64)
Closest candidates are:
  f(::Any, !Matched::Any) at In[11]:1

In [45]:
InverseGamma(-1.5,-3.2,check_args=false)

ArgumentError: ArgumentError: InverseGamma: the condition α > zero(α) && θ > zero(θ) is not satisfied.

In [44]:
Gamma(-1.5,-3.2,check_args=false)

Gamma{Float64}(α=-1.5, θ=-3.2)

In [35]:
InverseGamma(0.5,3.2)

InverseGamma{Float64}(invd=Gamma{Float64}(α=0.5, θ=0.3125), θ=3.2)

In [2]:
p = Normal(0,1)
differential_entropy(p)

1.4189385332046727

In [3]:
q = Normal(2,3)
kl_div(p,q)

0.8763900664458875

In [4]:
cross_entropy(p,q)

2.29532859965056

In [5]:
p2 = Normal(0,1)
q2 = Normal(2,3)

Normal{Float64}(μ=2.0, σ=3.0)

In [6]:
differential_entropy.([p,p2])

2-element Array{Float64,1}:
 1.4189385332046727
 1.4189385332046727

In [7]:
kl_div.([p,p2],[q,q2])

2-element Array{Float64,1}:
 0.8763900664458875
 0.8763900664458875

In [2]:
m1, m2 = Normal(0,2), Normal(2,1)
w1, w2 = Gamma(2,0.5), Gamma(3,1)
VMP(2.4,[m1,m2],[w1,w2])

2-element Array{Float64,1}:
 0.002647551691759494
 0.11107536260545436

In [9]:
collide(Gamma(2,0.5),Gamma(4,0.5), Gamma(1,5))

Gamma{Float64}(α=5.0, θ=4.2)

In [10]:
#p3 = MvNormal([2.3,-1.6],[1.0 0.001;0.001 1.0])
p3 = MvNormal([2.3,-1.6],diagm(0=>ones(2)))
println(differential_entropy(p3))
println(entropy(p3))

2.8378770664093453
2.8378770664093453


In [11]:
p4 = MvNormal([1.3,-1.5],[2.5 1.9;1.9 2.5])
println(differential_entropy(p3))
println(entropy(p3))

2.8378770664093453
2.8378770664093453


In [12]:
kl_div(p3,p4)

0.9825485494882034

In [13]:
h_p3, T_p3, η_p3, A_eval_p3, A_p3 = exp_family(p3);
h_p4, T_p4, η_p4, A_eval_p4, A_p4 = exp_family(p4);

In [14]:
using LinearAlgebra
kl(m1,m2,V1,V2,d) = 0.5*(logdet(V2)-logdet(V1) -d +tr(V2\V1) +(m2-m1)'*(V2\(m2-m1)))

kl (generic function with 1 method)

In [15]:
m3, m4, V3, V4 = [2.3,-1.6], [1.3,-1.5], [1 0.0;0.0 1], [2.5 1.9;1.9 2.5]

([2.3, -1.6], [1.3, -1.5], [1.0 0.0; 0.0 1.0], [2.5 1.9; 1.9 2.5])

In [16]:
kl(m3,m4,V3,V4,2)

0.9825485494882031

In [17]:
kl(m4,m3,V4,V3,2)

1.5196105414208874

In [18]:
kl_div(p4,p3)

1.5196105414208867

In [19]:
kl_div(p3,p4)

0.9825485494882034

In [20]:
q*2.4

Normal{Float64}(μ=4.8, σ=7.2)

In [21]:
1.5*q

Normal{Float64}(μ=3.0, σ=4.5)

In [22]:
A = 10*randn(3,2);
A*p3

LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [23]:
B = 5*randn(1,2);
B*p3

Normal{Float64}(μ=-0.9003246224998129, σ=7.153891176417836)

In [24]:
p + q

Normal{Float64}(μ=2.0, σ=3.1622776601683795)

In [25]:
p3+p4

FullNormal(
dim: 2
μ: [3.5999999999999996, -3.1]
Σ: [3.5 1.9; 1.9 3.5]
)


In [26]:
collide(p3,p4)

LoadError: MethodError: no method matching exp_family(::Type{MvNormal{Float64,PDMats.PDMat{Float64,Array{Float64,2}},Array{Float64,1}}}, ::Array{Float64,1})
Closest candidates are:
  exp_family(!Matched::Normal, ::Array{T,1} where T) at /Users/sakbayrak/Codes/BayesianUtilities/exp_family.jl:28
  exp_family(!Matched::MvNormal, ::Array{T,1} where T) at /Users/sakbayrak/Codes/BayesianUtilities/exp_family.jl:54
  exp_family(!Matched::Poisson, ::Array{T,1} where T) at /Users/sakbayrak/Codes/BayesianUtilities/exp_family.jl:103
  ...

In [27]:
# Product of two multivariate Gaussian messages for testing
function prod(p1::MvNormal, p2::MvNormal)
    S = cov(p1)+cov(p2)
    L = cholesky(S).L
    Linv = inv(L)
    S1 = Linv*cov(p1)
    S2 = Linv*cov(p2)
    m1 = Linv*mean(p1)
    m2 = Linv*mean(p2)
    S3 = Matrix(Hermitian(S1'*S2))
    m3 = S2'*m1 + S1'*m2
    return MvNormal(m3,S3)
end

prod (generic function with 1 method)

In [28]:
prod(p3,p4)

FullNormal(
dim: 2
μ: [1.872916666666667, -1.3395833333333336]
Σ: [0.5949074074074074 0.21990740740740738; 0.21990740740740738 0.5949074074074076]
)


In [29]:
diff_Ent(λ::Vector) = differential_entropy(exp_family(p,λ))

diff_Ent (generic function with 1 method)

In [30]:
diff_Ent([2,-4])

0.3792177623647548

In [38]:
Poisson(7; -)

LoadError: MethodError: no method matching Poisson(::Int64; -=-)
Closest candidates are:
  Poisson(::Integer) at /Users/sakbayrak/.julia/packages/Distributions/nAJyo/src/univariate/discrete/poisson.jl:34 got unsupported keyword argument "-"
  Poisson(::T; check_args) where T<:Real at /Users/sakbayrak/.julia/packages/Distributions/nAJyo/src/univariate/discrete/poisson.jl:29 got unsupported keyword argument "-"
  Poisson(::Any; -) at /Users/sakbayrak/Codes/BayesianUtilities/inference_rules.jl:32
  ...

In [31]:
exp_prob(x) = exp_family(p,x)

exp_prob (generic function with 1 method)

In [32]:
exp_prob(0.5)

0.3520653267642995

In [33]:
p5(λ) = Poisson(λ)
exp_prob(λ) = exp_family(p5(λ),3) 

exp_prob (generic function with 1 method)

In [34]:
exp_prob(4)

0.19536681481316448

In [35]:
p6 = Poisson(5)
deneme(η) = exp_family(p6,7,η)

deneme (generic function with 1 method)

In [36]:
using Zygote
deneme'(ones(2))

LoadError: DimensionMismatch("first array has length 2 which does not match the length of the second, 1.")

In [38]:
x = deneme'(ones(2))

In [39]:
x

In [40]:
deneme(ones(2))

0.104444862957054

In [43]:
using ForwardDiff
ForwardDiff.gradient(deneme,ones(2))

2-element Array{Float64,1}:
 0.0
 0.0

In [40]:
dump(:(exp_family(p6,x)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol exp_family
    2: Symbol p6
    3: Symbol x


In [41]:
dump(:(y*log(z) - z - log(y!)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol -
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol y
            3: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol log
                2: Symbol z
        3: Symbol z
    3: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol log
        2: Symbol y!


In [42]:
cici = :(y*log(z) - z - log(y!))

:((y * log(z) - z) - log(y!))

In [44]:
cici.args

3-element Array{Any,1}:
 :-
 :(y * log(z) - z)
 :(log(y!))

In [46]:
cici.args[2].args

3-element Array{Any,1}:
 :-
 :(y * log(z))
 :z

In [47]:
cici.args[2].args[2].args

3-element Array{Any,1}:
 :*
 :y
 :(log(z))

In [51]:
for a in cici.args
    @show a
end

a = :-
a = :(y * log(z) - z)
a = :(log(y!))


In [53]:
A_p3'(η_p3)

6-element Array{Float64,1}:
  2.3
 -1.6
  6.289999999999999
 -3.6799999999999997
 -3.6799999999999997
  3.5600000000000005